In [41]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import warnings
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
from math import sqrt
from datetime import datetime,timedelta
from dateutil.rrule import rrule, HOURLY
from statsmodels.tsa.arima.model import ARIMA
import itertools
import math
import time

from keras.models import Model, Sequential
from keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping

#warnings.filterwarnings("ignore")

def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res    

def mae(actual, predicted):
    res = np.empty(actual.shape)
    r = 0
    n = 0
    for j in range(actual.shape[0]):
            r += abs(actual[j] - predicted[j])
            n+=1
    return r/n    

def mse(actual, predicted):
    res = np.empty(actual.shape)
    r = 0
    n=0
    for j in range(actual.shape[0]):
            r += (actual[j] - predicted[j])**2
            n+=1
    return r/n    

def rmse (actual, predicted):
    r = mse(actual,predicted)
    return math.sqrt(r)

totalmae=0
totalmse=0
totalrmse=0
totalduration=0

frames=[]
totalmape=0
no_predictions = 24
lag = 6
batch_size=72
epochs = 100
patience = 50

#ids = [28079024, 28079038, 28079008, 28079040, 28079036, 28079018, 28079011, 28079004, 28079016, 28079039, 28079027]
#ids = [28079024, 28079038, 28079008, 28079047, 28079050, 28079048, 28079099, 28079026, 28079006, 28079022, 28079001, 28079015]
ids = [28079008, 28079047, 28079026, 28079006, 28079022, 28079015]
for k in ids:
    start_exec = time.time()
    dffinal=pd.read_csv("processed-"+str(k)+".csv", index_col='date',parse_dates=True)
    #dffinal=pd.read_csv("cleaned.csv", index_col='date',parse_dates=True)

    dffinal=dffinal.drop(columns=['NO','CH4','BEN','CO','EBE','MXY','NMHC','O_3','NOx','OXY','PM10','PXY','SO_2','TCH','TOL','NO_2','station'])
    dffinal.dropna()
    dffinal.drop(dffinal.index[365*24*2:],inplace=True)


    sc_in = MinMaxScaler(feature_range=(0, 1))
    dffinal['PM25'] = sc_in.fit_transform(dffinal[['PM25']])
    
    #split data
    train_size=  int(len(dffinal) *0.6)
    valid_size = (int)((int(len(dffinal)) - train_size)/2)
    test_size = (int(len(dffinal)) - train_size-valid_size)

    train = dffinal[:train_size].dropna()
    valid = dffinal[train_size:train_size+valid_size].dropna()
    test = dffinal[train_size+valid_size:].dropna()

    print("Total data size:",len(dffinal))
    print("Train data size:",len(train))
    print("Valid data size:",len(valid))
    print("Test data size:",len(test))
    

    train_shifted = train.copy();
    valid_shifted = valid.copy();
    test_shifted = test.copy();
    train_shifted['y_t+1'] = train_shifted['PM25'].shift(-1, freq='H')
    valid_shifted['y_t+1'] = valid_shifted['PM25'].shift(-1, freq='H')
    test_shifted['y_t+1'] = test_shifted['PM25'].shift(-1, freq='H')
    for t in range(1,lag+1):
        train_shifted[str(lag-t)] = train_shifted['PM25'].shift(lag-t, freq='H')
        valid_shifted[str(lag-t)] = valid_shifted['PM25'].shift(lag-t, freq='H')
        test_shifted[str(lag-t)] = test_shifted['PM25'].shift(lag-t, freq='H')
    y_col = 'y_t+1';
    X_cols = []
    for i in range(1,lag):
        X_cols.append('PM25_t-'+str(lag-i))
    X_cols.append('PM25_t')
    
    train_shifted.columns=['PM25']+[y_col]+X_cols
    train_shifted.dropna(how='any', inplace=True)
    valid_shifted.columns=['PM25']+[y_col]+X_cols
    valid_shifted.dropna(how='any', inplace=True)
    test_shifted.columns=['PM25']+[y_col]+X_cols
    test_shifted.dropna(how='any', inplace=True)

   
    y_train = train_shifted[y_col].values
    X_train = train_shifted[X_cols].values 
    X_train = X_train.reshape(X_train.shape[0],lag, 1)

    y_valid = valid_shifted[y_col].values
    X_valid = valid_shifted[X_cols].values 
    X_valid = X_valid.reshape(X_valid.shape[0],lag, 1)

    y_test = test_shifted[y_col].values
    X_test = test_shifted[X_cols].values 
    X_test = X_test.reshape(X_test.shape[0],lag, 1)
    
    #fit best model
    #model = Sequential();
    #model.add(GRU(lag, input_shape=(lag,1)))
    #model.add(Dense(no_predictions))
    #model.compile(optimizer='RMSprop', loss='mse')
    #model.summary()
    
    model = Sequential();
    model.add(GRU(300, dropout=0.5, input_shape=(lag,1)))
    model.add(Dense(units = no_predictions)) 
    #Compile model
    model.compile(optimizer='adam',loss='mse')
    model.summary();
    
    #model = Sequential();
    #model.add(GRU(lag, input_shape=(lag, 1)))
    #model.add(Dense(units = 24*24*100, activation = 'relu'))
    #model.add(Dropout(0.2))
    #model.add(Dense(units = 24, activation = 'linear'))
    #model.compile(loss='mean_squared_error', optimizer=RMSprop(lr = 0.00001))
    #model.summary()
    
    #model = Sequential()
    # First GRU layer with Dropout regularisation
    #model.add(GRU(units=120, return_sequences=True, input_shape=(lag,1), activation='tanh'))
    #model.add(Dropout(0.9))
    # Second GRU layer
    #model.add(GRU(units=120, return_sequences=True, input_shape=(lag,1), activation='tanh'))
    #model.add(Dropout(0.9))
    # Third GRU layer
    #model.add(GRU(units=120, return_sequences=True, input_shape=(lag,1), activation='tanh'))
    #model.add(Dropout(0.2))
    # Fourth GRU layer
    #model.add(GRU(units=lag, activation='tanh'))
    #model.add(Dropout(0.9))
    # The output layer
    #model.add(Dense(units=no_predictions))
    #model.compile(loss='mean_squared_error', optimizer=RMSprop(lr = 0.00001))
    #model.summary()
    
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=patience, restore_best_weights=True)
#    early_stop = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=patience)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_valid, y_valid),callbacks=[early_stop], verbose=1)
    #X_test = X_test[:1]
    #print(X_test)
    #print(test_shifted.head(24))
    forecast = model.predict(X_test)
    #print("Forecast",no_predictions,":")
    #print(forecast)

    evaluate = pd.DataFrame(forecast, columns=['t+'+str(i) for i in range(1,no_predictions+1)])
    evaluate['timestamp'] = test_shifted.index
    evaluate = pd.melt(evaluate, id_vars='timestamp',value_name='prediction', var_name='h')
    evaluate[['ht','hh']]=evaluate['h'].str.split('+',1, expand=True)
    evaluate['hh'] = pd.to_numeric(evaluate['hh'])
    evaluate['hh'] = pd.to_timedelta(evaluate['hh'], unit='h')
    evaluate['ts']=evaluate['timestamp']+evaluate['hh']
    mg = evaluate.merge(test_shifted, how='inner', left_on='ts', right_on='date')
    evaluate=mg[['timestamp','ts','h','prediction','PM25']];
    evaluate['diff'] = evaluate['prediction']-evaluate['PM25']
    evaluate['diff'] = evaluate['diff'].abs();
    evaluate['mape'] = evaluate['diff']/evaluate['PM25'];
    tmape = evaluate['mape'].sum()/evaluate['mape'].count();
    #print("TOTAL MAPE:",tmape)
    #print(evaluate)
    evaluate['mape']=abs((evaluate['PM25']-evaluate['prediction'])/evaluate['PM25'])
    vmape = np.mean(np.abs(percentage_error(np.asarray(evaluate['PM25']), np.asarray(evaluate['prediction'])))) * 100
    vmae = mae(np.asarray(evaluate['PM25']), np.asarray(evaluate['prediction']))
    vmse = mse(np.asarray(evaluate['PM25']), np.asarray(evaluate['prediction']))
    vrmse = rmse(np.asarray(evaluate['PM25']), np.asarray(evaluate['prediction']))
    end_exec = time.time()
    duration = end_exec-start_exec
    print("MAPE",k,'-',vmape);
    print("MAE",k,'-',vmae);
    print("MSE",k,'-',vmse);
    print("RMSE",k,'-',vrmse);
    print("duration",k,duration)
    totalmape += vmape
    totalmae += vmae
    totalmse += vmse
    totalrmse += vrmse
    totalduration += duration
    #fig, ax = plt.subplots(1, 1)
    #forecasts.plot(title="Forecasts", ax=ax)
#    forecasts.plot(kind='kde', title='Forecast', ax=ax[1])
#    plt.show();

#    predictions = model.predict(start=train_size, end=train_size+no_predictions-1)
#    print("Type predictions:",type(predictions))
#    pred = pd.DataFrame(predictions)
#    pred.index = list(pred.index)
#    print("Index pred:",pred.index)
#    print("Index test:",test_X.index)
#    a = pred.loc[train_size]
#    b = test_X.loc[train_size]
#    print("A:",a)
#    print("B:",b)
#    print("Type pred:",type(pred))
#    print("Predictions:")
#    print(pred);
#    tape = 0
#    for i in range(train_size, train_size+no_predictions-1):
#        ape = abs(pred.loc[i].predicted_mean-test_X.loc[i].PM25)/abs(test_X.loc[i].PM25)*100
#        tape = tape + ape
#    mape = tape/no_predictions;
#    print("MAPE:",mape)
#    print("Accuracy:",100.0-mape)
#    totalmape = totalmape + mape
overallmape = totalmape/len(ids)
overallmae = totalmae/len(ids)
overallmse = totalmse/len(ids)
overallrmse = totalrmse/len(ids)
overallduration = totalduration/len(ids)
print("Overall MAPE:",overallmape)
print("Overall accuracy:",100.0-overallmape)
print("Overall MAE:",overallmae)
print("Overall MSE:",overallmse)
print("Overall RMSE:",overallrmse)
print("Overall duration:",overallduration)

Total data size: 17520
Train data size: 10512
Valid data size: 3504
Test data size: 3504
Model: "sequential_193"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_227 (GRU)                (None, 300)               272700    
_________________________________________________________________
dense_193 (Dense)            (None, 24)                7224      
Total params: 279,924
Trainable params: 279,924
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
146/146 [==============================] - 3s 9ms/step - loss: 0.0068 - val_loss: 0.0019
Epoch 2/100
146/146 [==============================] - 1s 7ms/step - loss: 0.0060 - val_loss: 0.0026
Epoch 3/100
146/146 [==============================] - 1s 5ms/step - loss: 0.0059 - val_loss: 0.0017
Epoch 4/100
146/146 [==============================] - 1s 4ms/step - loss: 0.0057 - val_loss: 0.0019
Epoch 5/100
146

C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluate['diff'] = evaluate['prediction']-evaluate['PM25']
C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluate['diff'] = evaluate['diff'].abs();
C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

MAPE 28079008 - 60.44161062920692
MAE 28079008 - 0.05149841077438395
MSE 28079008 - 0.005092000582520427
RMSE 28079008 - 0.0713582551813063
duration 28079008 38.16134858131409
Total data size: 17520
Train data size: 10512
Valid data size: 3504
Test data size: 3504
Model: "sequential_194"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_228 (GRU)                (None, 300)               272700    
_________________________________________________________________
dense_194 (Dense)            (None, 24)                7224      
Total params: 279,924
Trainable params: 279,924
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
146/146 [==============================] - 2s 8ms/step - loss: 0.0064 - val_loss: 0.0022
Epoch 2/100
146/146 [==============================] - 1s 6ms/step - loss: 0.0057 - val_loss: 0.0020
Epoch 3/100
146/146 [====================

C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluate['diff'] = evaluate['prediction']-evaluate['PM25']
C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluate['diff'] = evaluate['diff'].abs();
C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

MAPE 28079047 - 57.91910311803691
MAE 28079047 - 0.05615625634741269
MSE 28079047 - 0.006990325288420076
RMSE 28079047 - 0.08360816520185141
duration 28079047 37.33326721191406
Total data size: 17520
Train data size: 10512
Valid data size: 3504
Test data size: 3504
Model: "sequential_195"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_229 (GRU)                (None, 300)               272700    
_________________________________________________________________
dense_195 (Dense)            (None, 24)                7224      
Total params: 279,924
Trainable params: 279,924
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
146/146 [==============================] - 2s 6ms/step - loss: 0.0069 - val_loss: 0.0043
Epoch 2/100
146/146 [==============================] - 1s 5ms/step - loss: 0.0054 - val_loss: 0.0044
Epoch 3/100
146/146 [===================

...

C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluate['diff'] = evaluate['prediction']-evaluate['PM25']
C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluate['diff'] = evaluate['diff'].abs();
C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

MAPE 28079006 - 45.50269720621774
MAE 28079006 - 0.05195342674437734
MSE 28079006 - 0.0053500285262020825
RMSE 28079006 - 0.07314388919248198
duration 28079006 40.1088285446167
Total data size: 17520
Train data size: 10512
Valid data size: 3504
Test data size: 3504
Model: "sequential_197"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_231 (GRU)                (None, 300)               272700    
_________________________________________________________________
dense_197 (Dense)            (None, 24)                7224      
Total params: 279,924
Trainable params: 279,924
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
146/146 [==============================] - 2s 6ms/step - loss: 0.0133 - val_loss: 0.0042
Epoch 2/100
146/146 [==============================] - 1s 4ms/step - loss: 0.0114 - val_loss: 0.0045
Epoch 3/100
146/146 [===================

C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluate['diff'] = evaluate['prediction']-evaluate['PM25']
C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluate['diff'] = evaluate['diff'].abs();
C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

MAPE 28079022 - 75.98921595946678
MAE 28079022 - 0.08998482603307738
MSE 28079022 - 0.016463057212724824
RMSE 28079022 - 0.12830844560170163
duration 28079022 36.793288469314575
Total data size: 17520
Train data size: 10512
Valid data size: 3504
Test data size: 3504
Model: "sequential_198"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_232 (GRU)                (None, 300)               272700    
_________________________________________________________________
dense_198 (Dense)            (None, 24)                7224      
Total params: 279,924
Trainable params: 279,924
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
146/146 [==============================] - 2s 6ms/step - loss: 0.0074 - val_loss: 0.0049
Epoch 2/100
146/146 [==============================] - 1s 4ms/step - loss: 0.0044 - val_loss: 0.0046
Epoch 3/100
146/146 [==================

C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluate['diff'] = evaluate['prediction']-evaluate['PM25']
C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluate['diff'] = evaluate['diff'].abs();
C:\Users\Johnny\AppData\Local\Temp\ipykernel_47812\3948869189.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r